In [1]:
import pandas as pd
import os
import csv
from csv import reader
import lxml.etree as ET
import sqlite3
import time

import urllib.parse
import urllib.request
import urllib.error

In [2]:
HPB = "https://sru.gbv.de/hpb?version=2.0&operation=searchRetrieve&query="
conn = sqlite3.connect(os.path.join('stcv.sqlite'))
c = conn.cursor()

In [3]:
query = """
SELECT DISTINCT
    title.cloi as identifier,
    COUNT(title.cloi) as id_count,
    author_vw as author_standardized,
    author_zvwr as author_original,
    author_zbd as author_dates,
    corporateauthor_nm as corporateauthor_standardized,
    corporateauthor_zvwr as corporateauthor_original,
    title_ti as title_title,
    title_lg as title_language,
    collation_fm as format,
    collation_ka as quires,
    collation_pg as pages,
    edition_ed as edition_info,
    impressum_ju1sv as year1,
    impressum_ju1ty as year1_type,
    impressum_ju2sv as year2,
    impressum_ju2ty as year2_type,
    impressum_pl as place,
    impressum_ug as printer,
    language_lg as language_info,
    number_nr as fingerprint
    FROM title
LEFT JOIN author on author.cloi = title.cloi
LEFT JOIN collation on collation.cloi = title.cloi
LEFT JOIN corporateauthor on corporateauthor.cloi = title.cloi
LEFT JOIN edition on edition.cloi = title.cloi
LEFT JOIN impressum on impressum.cloi = title.cloi
LEFT JOIN language on language.cloi = title.cloi
LEFT JOIN number on number.cloi = title.cloi
GROUP BY identifier
HAVING ID_COUNT=2
LIMIT 100
"""

In [4]:
c.execute(query)

data = [row for row in c.fetchall()]

FILENAME = "mycsvfile.csv"

with open(FILENAME, 'w', encoding="utf-8") as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerows(data)

conn.close()

In [5]:
df = pd.read_csv(FILENAME, header=None)

df.to_csv(FILENAME, header=["identifier", "id_count", "author_standardized", "author_original", "author_dates",
                                   "corporateauthor_standardized", "corporateauthor_original", "title_title", "title_language",
                                   "format", "quires", "pages", "edition_info", "year1", "year1_type", "year2", "year2_type",
                                   "place", "printer", "language_info", "fingerprint"], index=False)

In [6]:
def search_param(row: list):
    author = row[2]
    title = row[7]
    year = row[13]
    identifier = row[0]
    return author, title, year, identifier

    
def form_url(author, title, year):
    author = urllib.parse.quote(author)
    title = urllib.parse.quote(title)
    url = HPB + author + "%20\"" + title + "\"%20" + year + "&startRecord=1&maximumRecords=10"
    return url


def form_url_alt(title, year):
    title = urllib.parse.quote(title)
    url = HPB + "%20\"" + title + "\"%20" + year + "&startRecord=1&maximumRecords=10"
    return url


def query_HPB(search: str) -> bytes:
    try:
        with urllib.request.urlopen(url) as query:
            return query.read()
    except urllib.error.HTTPError as HTTPerr:
        exit(HTTPerr.code)
    except urllib.error.URLError as URLerr:
        exit(URLerr)      
        
        
def matcher(reply):
    tree = ET.ElementTree(ET.fromstring(reply, parser = ET.XMLParser()))
    root = tree.getroot()
    for elem in root.iter():
        if 'datafield' in elem.tag:
            if elem.attrib['tag'] == '035':
                grandchildren = elem.getchildren()
                for grandchild in grandchildren:
                    if grandchild.attrib['code'] == 'a':
                        return grandchild.text

In [7]:
with open(FILENAME, 'r', encoding="utf-8") as file:
    query_file = csv.reader(file)
    next(query_file, None)
    n = 1
    for row in query_file:
        url = form_url(search_param(row)[0], search_param(row)[1], search_param(row)[2])
        reply = query_HPB(url)
        hpb_id = matcher(reply)
        identifier = search_param(row)[3]
        try:
            print(str(n) + ". " + identifier + " matches " + hpb_id)
        except TypeError:
            print(str(n) + ". The primary search didn't yeild any results. Trying the alternative method...")
            url = form_url_alt(search_param(row)[1], search_param(row)[2])
            reply = query_HPB(url)
            hpb_id = matcher(reply)
            identifier = search_param(row)[3]
            print("   Secondary search results:\n    " + identifier + " matches " + hpb_id)
        n += 1
        time.sleep(4)

1. c:stcv:12854444 matches BE-AnVE.c:stcv:12854444
2. c:stcv:12854501 matches BE-AnVE.c:stcv:12854501
3. c:stcv:12854549 matches BE-AnVE.c:stcv:12854549
4. c:stcv:12854587 matches BE-AnVE.c:stcv:12854587
5. c:stcv:12854599 matches BE-AnVE.c:stcv:12854599
6. c:stcv:12857052 matches BE-AnVE.c:stcv:12857052
7. c:stcv:12857069 matches BE-AnVE.c:stcv:12857069
8. c:stcv:12857131 matches BE-AnVE.c:stcv:12857131
9. c:stcv:12857158 matches BE-AnVE.c:stcv:12923714
10. c:stcv:12857275 matches BE-AnVE.c:stcv:12857275
11. c:stcv:12857374 matches BE-AnVE.c:stcv:12857374
12. c:stcv:12857912 matches BE-AnVE.c:stcv:12857912
13. c:stcv:12857923 matches BE-AnVE.c:stcv:12857923
14. c:stcv:12858124 matches BE-AnVE.c:stcv:12858124
15. c:stcv:12858392 matches BE-AnVE.c:stcv:12858392
16. c:stcv:12858408 matches BE-AnVE.c:stcv:12858408
17. c:stcv:12858413 matches BE-AnVE.c:stcv:12858413
18. c:stcv:12858468 matches BE-AnVE.c:stcv:12858468
19. c:stcv:12858494 matches BE-AnVE.c:stcv:12858509
20. c:stcv:12858622 m